In [67]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import os
import time
import warnings
import os
from six.moves import urllib
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('seaborn')
from scipy.stats import norm, skew

In [68]:
#Add All the Models Libraries

# Scalers
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

# Models

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error

#regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from scipy.stats import reciprocal, uniform

# Cross-validation
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.model_selection import cross_validate

# GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Common data processors
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from scipy import sparse

In [69]:
# to make this notebook's output stable across runs
np.random.seed(123)

# To plot pretty figures
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [70]:
train = pd.read_csv('U:\\Elo_Merchant\\train.csv',parse_dates=["first_active_month"])
test = pd.read_csv('U:\\Elo_Merchant\\test.csv', parse_dates=["first_active_month"])
merchants = pd.read_csv('U:\\Elo_Merchant\\merchants.csv')

In [71]:
new_transactions_raw = pd.read_csv('U:\\Elo_Merchant\\new_merchant_transactions.csv')

In [72]:
transactions_raw = pd.read_csv('U:\\Elo_Merchant\\historical_transactions.csv')

In [73]:
train.head(2)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913


<pre>
first_active_month  : ''YYYY-MM', month of first purchase
feature_1,2,3       : Anonymized card categorical feature
target              : Loyalty numerical score calculated 2 months after historical and evaluation period</pre>

In [74]:
train.shape

(201917, 6)

In [75]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
first_active_month    201917 non-null datetime64[ns]
card_id               201917 non-null object
feature_1             201917 non-null int64
feature_2             201917 non-null int64
feature_3             201917 non-null int64
target                201917 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 9.2+ MB


In [76]:
test.shape

(123623, 5)

In [77]:
merchants.shape

(334696, 22)

In [78]:
merchants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 22 columns):
merchant_id                    334696 non-null object
merchant_group_id              334696 non-null int64
merchant_category_id           334696 non-null int64
subsector_id                   334696 non-null int64
numerical_1                    334696 non-null float64
numerical_2                    334696 non-null float64
category_1                     334696 non-null object
most_recent_sales_range        334696 non-null object
most_recent_purchases_range    334696 non-null object
avg_sales_lag3                 334683 non-null float64
avg_purchases_lag3             334696 non-null float64
active_months_lag3             334696 non-null int64
avg_sales_lag6                 334683 non-null float64
avg_purchases_lag6             334696 non-null float64
active_months_lag6             334696 non-null int64
avg_sales_lag12                334683 non-null float64
avg_purchases_lag12    

In [79]:
merchants.head(2)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0


In [80]:
# Now extract the month, year, day, weekday
train["month"] = train["first_active_month"].dt.month
train["year"] = train["first_active_month"].dt.year
train['days'] = (datetime.date(2018, 2, 1) - train['first_active_month'].dt.date).dt.days

test["month"] = train["first_active_month"].dt.month
test["year"] = train["first_active_month"].dt.year
test['days'] = (datetime.date(2018, 2, 1) - test['first_active_month'].dt.date).dt.days

In [81]:
train = pd.get_dummies(train, columns=['feature_1', 'feature_2'])
test = pd.get_dummies(test, columns=['feature_1', 'feature_2'])

In [82]:
train.head(2)

,first_active_month,card_id,feature_3,target,month,year,days,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3
0,2017-06-01,C_ID_92a2005557,1,-0.820283,6,2017,245,0,0,0,0,1,0,1,0
1,2017-01-01,C_ID_3d0044924f,0,0.392913,1,2017,396,0,0,0,1,0,1,0,0


Now we will try to extract more features from Transactions Data

In [83]:
transactions = pd.get_dummies(transactions_raw, columns=['category_2', 'category_3'])
transactions['authorized_flag'] = transactions_raw['authorized_flag'].map({'Y': 1, 'N': 0})
transactions['category_1'] = transactions_raw['category_1'].map({'Y': 1, 'N': 0})
transactions.head(2)

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,1,0,0,0,0,1,0,0
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,1,0,0,0,0,1,0,0


In [84]:
# Taking Reference from Other Kernels
def aggregate_transactions(trans, prefix):  
    trans.loc[:, 'purchase_date'] = pd.DatetimeIndex(trans['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp],
        'month_lag': ['min', 'max']
    }
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() 
                           for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    
    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))
    
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    
    return agg_trans

In [85]:
import gc
merge_trans = aggregate_transactions(transactions, prefix='hist_')
#del transactions
gc.collect()

train = pd.merge(train, merge_trans, on='card_id',how='left')
test = pd.merge(test, merge_trans, on='card_id',how='left')
#del merge_trans
gc.collect()

train.head(2)

,first_active_month,card_id,feature_3,target,month,year,days,feature_1_1,feature_1_2,feature_1_3,...,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_date_ptp,hist_month_lag_min,hist_month_lag_max
0,2017-06-01,C_ID_92a2005557,1,-0.820283,6,2017,245,0,0,0,...,-0.739395,0.212139,4,0.015385,1,0,0.123314,20977987.0,-8,0
1,2017-01-01,C_ID_3d0044924f,0,0.392913,1,2017,396,0,0,0,...,-0.742400,0.384967,543,1.551429,10,-1,1.510777,33717687.0,-12,0


In [86]:
# Now extract the data from the new transactions

new_transactions = pd.get_dummies(new_transactions_raw, columns=['category_2', 'category_3'])
new_transactions['authorized_flag'] = new_transactions_raw['authorized_flag'].map({'Y': 1, 'N': 0})
new_transactions['category_1'] = new_transactions_raw['category_1'].map({'Y': 1, 'N': 0})
new_transactions.head(2)

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0
1,1,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0


In [87]:
merge_new = aggregate_transactions(new_transactions, prefix='new_')
#del new_transactions
gc.collect()

train = pd.merge(train, merge_new, on='card_id',how='left')
test = pd.merge(test, merge_new, on='card_id',how='left')
#del merge_new

gc.collect()
train.head(2)

,first_active_month,card_id,feature_3,target,month,year,days,feature_1_1,feature_1_2,feature_1_3,...,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_date_ptp,new_month_lag_min,new_month_lag_max
0,2017-06-01,C_ID_92a2005557,1,-0.820283,6,2017,245,0,0,0,...,-0.724368,0.135812,0.0,0.0,0.0,0.0,0.0,4742309.0,1.0,2.0
1,2017-01-01,C_ID_3d0044924f,0,0.392913,1,2017,396,0,0,0,...,-0.739410,0.014326,6.0,1.0,1.0,1.0,0.0,4887632.0,1.0,2.0


In [88]:
# Delete the datasources once the merge is done
del new_transactions
del merge_new
del transactions
del merge_trans
del new_transactions_raw
del transactions_raw

In [89]:
# Now check the shape of Train and Test Data
train.shape

(201917, 67)

In [90]:
test.shape

(123623, 66)

In [91]:
train = train.drop(['card_id','first_active_month'], axis=1)
test = test.drop(['card_id','first_active_month'], axis=1)

In [95]:
categoryVariableList = ['month','year']
for var in categoryVariableList:
    train[var] = train[var].astype("category")
    test[var] = test[var].astype("category")

In [98]:
# Get the correlation matrix for the training set

train_y = train.target.reset_index(drop=True)
train_x = train.drop(['target'], axis=1)
test_x = test

Data Transformation and Imputations

In [105]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [117]:
columns = [ 'hist_transactions_count',
            'hist_authorized_flag_sum',
            'hist_authorized_flag_mean',
            'hist_category_1_mean',
            'hist_category_2_1.0_mean',
            'hist_category_2_2.0_mean',
            'hist_category_2_3.0_mean',
            'hist_category_2_4.0_mean',
            'hist_category_2_5.0_mean',
            'hist_category_3_A_mean',
            'hist_category_3_B_mean',
            'hist_category_3_C_mean',
            'hist_merchant_id_nunique',
            'hist_purchase_amount_sum',
            'hist_purchase_amount_mean',
            'hist_purchase_amount_max',
            'hist_purchase_amount_min',
            'hist_purchase_amount_std',
            'hist_installments_sum',
            'hist_installments_mean',
            'hist_installments_max',
            'hist_installments_min',
            'hist_installments_std',
            'hist_purchase_date_ptp',
            'hist_month_lag_min',
            'hist_month_lag_max',
            'new_transactions_count',
            'new_authorized_flag_sum',
            'new_authorized_flag_mean',
            'new_category_1_mean',
            'new_category_2_1.0_mean',
            'new_category_2_2.0_mean',
            'new_category_2_3.0_mean',
            'new_category_2_4.0_mean',
            'new_category_2_5.0_mean',
            'new_category_3_A_mean',
            'new_category_3_B_mean',
            'new_category_3_C_mean',
            'new_merchant_id_nunique',
            'new_purchase_amount_sum',
            'new_purchase_amount_mean',
            'new_purchase_amount_max',
            'new_purchase_amount_min',
            'new_purchase_amount_std',
            'new_installments_sum',
            'new_installments_mean',
            'new_installments_max',
            'new_installments_min',
            'new_installments_std',
            'new_purchase_date_ptp',
            'new_month_lag_min',
            'new_month_lag_max',
            'days'
]

for col in columns:
    train_x.update(train_x[col].fillna(0, inplace=True))
    test_x.update(test_x[col].fillna(0, inplace=True))

In [118]:
#Check for missing values in training set
nulls = np.sum(train_x.isnull())
nullcols = nulls.loc[(nulls != 0)]
dtypes = train_x.dtypes
dtypes2 = dtypes.loc[(nulls != 0)]
info = pd.concat([nullcols, dtypes2], axis=1).sort_values(by=0, ascending=False)
print(info)
print("There are", len(nullcols), "columns with missing values in training set")

Empty DataFrame
Columns: [0, 1]
Index: []
There are 0 columns with missing values in training set


In [119]:
#Check for missing values in test set
nulls = np.sum(test_x.isnull())
nullcols = nulls.loc[(nulls != 0)]
dtypes = test_x.dtypes
dtypes2 = dtypes.loc[(nulls != 0)]
info = pd.concat([nullcols, dtypes2], axis=1).sort_values(by=0, ascending=False)
print(info)
print("There are", len(nullcols), "columns with missing values in test set")

Empty DataFrame
Columns: [0, 1]
Index: []
There are 0 columns with missing values in test set


In [ ]:
#Now let's use t-SNE to reduce dimensionality down to 2D so we can plot the dataset:

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42, verbose = 2, njobs = -1)
TSNE_X = tsne.fit_transform(train_x)
TSNE_X_test = tsne.fit_transform(test_x)